# Projet maison n° 3

In [71]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python `df_names_us()` qui produit un unique DataFrame avec tous les fichiers en utilisant **pandas** (par ex. avec la fonction `pandas.concat()`), pas de bash :)

Ordre et noms des colonnes du DataFrame : "year", "name", "gender", "births"

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini dans les différents fichiers (voir la documentation du dataset).

In [72]:
import pandas as pd
import glob

# names us
def df_names_us():
    
    df = pd.DataFrame()

    files = glob.glob('*.txt') #Get files
    frame = []
    year_list= []
    
    for file in files : #Store data in empty list
        data = pd.read_csv(file, sep=",", names=['name', 'gender', 'births'])
        frame.append(data)
    
        for k in range(len(data)): #Store year of births in another empty list
            year_list.append(file[3:7] )
    

    df = pd.concat(frame) #Concat files data in one dataframe
    df = df.assign(year=year_list) #add year column
    
    df = pd.DataFrame.reindex(df,columns = ['year', 'name', 'gender', 'births']) #Arrange dataframe
    df.sort_values(by=['year','gender', 'births'], inplace=True, ascending = [True, True, False]) #Sort dataframe
    df = df.drop_duplicates().reset_index(drop=True) #Reset index
    

    return df

In [73]:
df_names_us()


,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746
...,...,...,...,...
2052776,2021,Zyeire,M,5
2052777,2021,Zyel,M,5
2052778,2021,Zyian,M,5
2052779,2021,Zylar,M,5


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2021_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction `df_names_fr()` qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne "gender"
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : year (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [74]:
# names fr
def df_names_fr():
    
    dffr = pd.DataFrame()
    file = 'nat2021.csv' #File name
    dffr = pd.read_csv(file, sep=";", keep_default_na=False) #Get file

    #Same order and same columns names
    dffr.columns=['gender', 'name', 'year', 'births'] #Columns name
    dffr = pd.DataFrame.reindex(dffr,columns = ['year', 'name', 'gender', 'births']) #Arrange columns

    #Same dtypes (year:char, name:char, gender:char, births:int)
    dffr = dffr.astype({'year': str, 'name': str, 'births':int})

    #Same values for 'gender' columns
    dffr['gender'] = dffr['gender'].replace([1, 2],['M','F'])


    #Only name with 2 char or more are kept        
    dffr = dffr[ dffr['name'].apply( lambda x : len(str(x)) > 1 ) ]

    #Same name form
    dffr['name'] = dffr['name'].str.title()

    #Remove rows with unusable data
    dffr = dffr.drop(dffr[ ( dffr.year == 'XXXX' ) ].index) #Remove 'XXXX' year
    dffr = dffr.drop(dffr[ ( dffr.name == '_Prenoms_Rares' ) ].index) #Remove '_Prenom_Rares' name rows

    #Sort data
    dffr = dffr.sort_values(by = ['year', 'gender', 'births', 'name'], ascending = [True, True, False, True])
    dffr = dffr.reset_index(drop=True)
    
    
    return dffr


In [75]:
df_names_fr()

,year,name,gender,births
0,1900,Marie,F,48713
1,1900,Jeanne,F,13981
2,1900,Marguerite,F,8058
3,1900,Germaine,F,6981
4,1900,Louise,F,6696
...,...,...,...,...
648325,2021,Élyam,M,3
648326,2021,Énael,M,3
648327,2021,Éros,M,3
648328,2021,Étan,M,3


## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

Les données sont dans le fichier "Webstat_Export.csv".

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3. On retiendra uniquement les colonnes se terminant effectivement par un code ISO3 entre parenthèses. Par ex., "Dollar des Etats-Unis (USD)".

Implémenter une fonction `df_taux_change()` qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction `pd.to_datetime()` avec le format`"%d/%m/%Y"`) et doit être trié par ordre croissant. Les colonnes du DataFrame doivent correspondre aux devises sélectionnées.

In [118]:
# taux de change
def df_taux_change(devises):
    
    file = 'Webstat_Export.csv' #File name
    data = pd.read_csv(file, sep = ';')
    dftc = pd.DataFrame(data)


    dftc = dftc.drop(dftc.head(6).index) #Remove first rows


    dftc.rename(columns = {'Titre :':'date'}, inplace = True) #Rename date column
    dftc = dftc.set_index(pd.to_datetime(dftc["date"], infer_datetime_format = True)) #Set index with good form
    dftc = dftc.drop(columns = "date") #we don't need date column anymore

    dftc = dftc.sort_values(by = ["date"], ascending = True) #sort values by date

    dftc.rename(columns = ( lambda x : x[-4:-1] ), inplace=True) #Rename columns with ISO3 code


    #Clean dataframe
    dftc = dftc.apply( lambda x: x.str.replace(',', '.') )
    dftc = dftc.apply( lambda x: x.replace('-', None) )
    dftc = dftc.astype(float)

    dftc[devises]
    
    return dftc[devises]


In [124]:
devises = ['GBP','USD']
df_taux_change(devises)

,GBP,USD
date,,
1999-01-04,0.71110,1.1789
1999-01-05,0.71220,1.1790
1999-01-06,0.70760,1.1743
1999-01-07,0.70585,1.1632
1999-01-08,0.70940,1.1659
...,...,...
2022-09-15,0.86934,0.9992
2022-09-16,0.87400,0.9954
2022-09-17,0.87400,0.9954


### Tests

In [119]:
import unittest

class Session3Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 2052781)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 648330)
        # index
        self.assertTrue(isinstance(df.index, pd.RangeIndex))
        # test names
        self.assertTrue(df.loc[df.name.str.contains(r"^[A-Z]+$")].empty)
        self.assertTrue(df.loc[df.name.str.contains(r"-[a-z]+$")].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df.gender=="F"]) + len(df.loc[df.gender=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(["CHF", "GBP", "USD"])
        # colonnes
        self.assertEqual(list(df.columns), ["CHF", "GBP", "USD"])
        # index
        self.assertTrue(isinstance(df.index, pd.DatetimeIndex))
        # index trié
        self.assertEqual(list(df.index.argsort()), list(range(len(df))))
        # types taux
        self.assertTrue((df.dtypes == float).all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [120]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session3Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [121]:
# run tests
run_tests()

test_df_names_fr (__main__.Session3Tests) ... ok
test_df_names_us (__main__.Session3Tests) ... ok
test_df_taux_change (__main__.Session3Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 6.911s

OK
